# Indexing division hints

In generating an index over a catalog column, we use dask's `set_index` method to shuffle the catalog data around. This can be a very expensive operation. We can save a lot of time and general compute resources if we have some intelligent prior information about the distribution of the values inside the column we're building an index on.

In this notebook, I build some divisions for the ZTF DR14 data on the ID column. This uses the PanSTARRS object ID as the object ID, which is a large integer value.

See also:
* https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.set_index.html#dask.dataframe.DataFrame.set_index
* https://docs.dask.org/en/latest/dataframe-design.html#partitions
* https://docs.dask.org/en/stable/dataframe-best-practices.html#avoid-full-data-shuffling

In [1]:
from hipscat.io.parquet_metadata import write_parquet_metadata
from hipscat.io import file_io
import os

## Specify the catalog and column you're making your index over.
input_catalog_path="/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14"
indexing_column="ps1_objid"

We're going to look a lot into the `_metadata` file, which is a parquet file at the root of a hipscat catalog. Among other things, it contains statistics about the min and max values our data takes within each leaf parquet file.

Let's make sure that the indexing column really exists in our data. Then, we can visually check that it's correct by looking at the per-column schema in the parquet files.

In [2]:
## you might not need to change anything after that.
total_metadata = file_io.read_parquet_metadata(os.path.join(input_catalog_path, "_metadata"))

num_row_groups = total_metadata.num_row_groups

first_row_group = total_metadata.row_group(0)
index_column_idx = -1

for i in range(0, first_row_group.num_columns):
    column = first_row_group.column(i)
    if column.path_in_schema == indexing_column:
        index_column_idx = i
print("found column at index:", index_column_idx)

found column at index: 0


In [3]:
total_metadata.schema

required group field_id=-1 schema {
  optional int64 field_id=-1 ps1_objid;
  optional double field_id=-1 ra;
  optional double field_id=-1 dec;
  optional double field_id=-1 ps1_gMeanPSFMag;
  optional double field_id=-1 ps1_rMeanPSFMag;
  optional double field_id=-1 ps1_iMeanPSFMag;
  optional int32 field_id=-1 nobs_g;
  optional int32 field_id=-1 nobs_r;
  optional int32 field_id=-1 nobs_i;
  optional double field_id=-1 mean_mag_g;
  optional double field_id=-1 mean_mag_r;
  optional double field_id=-1 mean_mag_i;
  optional int32 field_id=-1 Norder;
  optional int32 field_id=-1 Dir;
  optional int32 field_id=-1 Npix;
  optional int64 field_id=-1 _hipscat_index (Int(bitWidth=64, isSigned=false));
}

We're making a guess that the `ps1_objid` is uniformly distributed.

First, find the minimum and maximum values across all of our data. We do this just by looking inside that `_metadata` file - we don't need to do a full catalog scan for these high-level statistics!

Then use those values, and a little arithmetic to create a **list** of divisions (it's important to dask that this be a list, and not a numpy array). Pass this list along to your `ImportArguments`!

In [4]:
import numpy as np

global_min = total_metadata.row_group(0).column(index_column_idx).statistics.min
global_max = total_metadata.row_group(0).column(index_column_idx).statistics.max

for index in range(1, num_row_groups):
    global_min = min(global_min, total_metadata.row_group(index).column(index_column_idx).statistics.min)
    global_max = max(global_max, total_metadata.row_group(index).column(index_column_idx).statistics.max)

print("global min", global_min)
print("global max", global_max)

increment = int((global_max-global_min)/num_row_groups)

divisions = np.append(np.arange(start = global_min, stop = global_max, step = increment), global_max)

divisions.tolist()

global min 71150119096299949
global max 215050952450164082


[71150119096299949,
 71211301423406183,
 71272483750512417,
 71333666077618651,
 71394848404724885,
 71456030731831119,
 71517213058937353,
 71578395386043587,
 71639577713149821,
 71700760040256055,
 71761942367362289,
 71823124694468523,
 71884307021574757,
 71945489348680991,
 72006671675787225,
 72067854002893459,
 72129036329999693,
 72190218657105927,
 72251400984212161,
 72312583311318395,
 72373765638424629,
 72434947965530863,
 72496130292637097,
 72557312619743331,
 72618494946849565,
 72679677273955799,
 72740859601062033,
 72802041928168267,
 72863224255274501,
 72924406582380735,
 72985588909486969,
 73046771236593203,
 73107953563699437,
 73169135890805671,
 73230318217911905,
 73291500545018139,
 73352682872124373,
 73413865199230607,
 73475047526336841,
 73536229853443075,
 73597412180549309,
 73658594507655543,
 73719776834761777,
 73780959161868011,
 73842141488974245,
 73903323816080479,
 73964506143186713,
 74025688470292947,
 74086870797399181,
 74148053124505415,


To avoid copy/pasting that giant list, you can just copy/paste the statement needed to generate the divisions:

In [5]:
print(f"np.append(np.arange(start = {global_min}, stop = {global_max}, step = {increment}), {global_max}).tolist()")

np.append(np.arange(start = 71150119096299949, stop = 215050952450164082, step = 61182327106234), 215050952450164082).tolist()
